# Bài tập Decision Tree

## Use Decision Tree to classify the Iris dataset
- dataset: 'Iris.csv'


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Iris.csv')

In [3]:

import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [4]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# chuyển columns Species thành 3 loại hoa tương ứng 0,1,2

In [5]:
for dataset in df:
    df['Species'] = df['Species'].replace('Iris-setosa', 0)
    df['Species'] = df['Species'].replace('Iris-versicolor', 1)
    df['Species'] = df['Species'].replace('Iris-virginica', 2)

df


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,2
146,147,6.3,2.5,5.0,1.9,2
147,148,6.5,3.0,5.2,2.0,2
148,149,6.2,3.4,5.4,2.3,2


In [6]:
# chuyển file iris thành DF với những chỉ số biểu hiện cho độ dài, độ rộng của hoa
data = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm', 'PetalWidthCm']].values
data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [7]:
target = df[['Species']].values
target

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [8]:
# chia tập test train thành 80-20 (dựa trên nguyên lý 80-20),
X_train, X_test, y_train, y_test = train_test_split( data, target, test_size = 0.2)

In [9]:
# Tạo ra decision tree
decision_tree = tree.DecisionTreeClassifier(max_depth=3)
#max_depth = 3 là độ sâu của decision tree
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [10]:

acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
print("train precision:" , acc_decision_tree)

# Độ chính xác 98.33% 

train precision: 98.33


In [11]:
y_pred = decision_tree.predict(X_test)
y_pred

array([2, 2, 1, 0, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 1, 0, 1, 1, 1, 2, 0, 2,
       1, 1, 0, 1, 1, 2, 0, 1])

In [12]:
from sklearn.tree import export_graphviz

dot_data = export_graphviz(decision_tree, out_file=None)
print(dot_data)

# copy code dưới vào trang http://viz-js.com/. sẽ ra được hình ảnh trực quan của Decision Tree

digraph Tree {
node [shape=box] ;
0 [label="X[3] <= 0.8\ngini = 0.666\nsamples = 120\nvalue = [43, 38, 39]"] ;
1 [label="gini = 0.0\nsamples = 43\nvalue = [43, 0, 0]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="X[3] <= 1.75\ngini = 0.5\nsamples = 77\nvalue = [0, 38, 39]"] ;
0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
3 [label="X[2] <= 5.35\ngini = 0.097\nsamples = 39\nvalue = [0, 37, 2]"] ;
2 -> 3 ;
4 [label="gini = 0.051\nsamples = 38\nvalue = [0, 37, 1]"] ;
3 -> 4 ;
5 [label="gini = 0.0\nsamples = 1\nvalue = [0, 0, 1]"] ;
3 -> 5 ;
6 [label="X[2] <= 4.85\ngini = 0.051\nsamples = 38\nvalue = [0, 1, 37]"] ;
2 -> 6 ;
7 [label="gini = 0.444\nsamples = 3\nvalue = [0, 1, 2]"] ;
6 -> 7 ;
8 [label="gini = 0.0\nsamples = 35\nvalue = [0, 0, 35]"] ;
6 -> 8 ;
}


In [13]:
import graphviz 
from graphviz import Digraph
# graph = graphviz.Source(dot_data)
# graph

dot = Digraph(dot_data)
dot.source

'digraph "digraph Tree {\nnode [shape=box] ;\n0 [label=\\"X[3] <= 0.8\\ngini = 0.666\\nsamples = 120\\nvalue = [43, 38, 39]\\"] ;\n1 [label=\\"gini = 0.0\\nsamples = 43\\nvalue = [43, 0, 0]\\"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel=\\"True\\"] ;\n2 [label=\\"X[3] <= 1.75\\ngini = 0.5\\nsamples = 77\\nvalue = [0, 38, 39]\\"] ;\n0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel=\\"False\\"] ;\n3 [label=\\"X[2] <= 5.35\\ngini = 0.097\\nsamples = 39\\nvalue = [0, 37, 2]\\"] ;\n2 -> 3 ;\n4 [label=\\"gini = 0.051\\nsamples = 38\\nvalue = [0, 37, 1]\\"] ;\n3 -> 4 ;\n5 [label=\\"gini = 0.0\\nsamples = 1\\nvalue = [0, 0, 1]\\"] ;\n3 -> 5 ;\n6 [label=\\"X[2] <= 4.85\\ngini = 0.051\\nsamples = 38\\nvalue = [0, 1, 37]\\"] ;\n2 -> 6 ;\n7 [label=\\"gini = 0.444\\nsamples = 3\\nvalue = [0, 1, 2]\\"] ;\n6 -> 7 ;\n8 [label=\\"gini = 0.0\\nsamples = 35\\nvalue = [0, 0, 35]\\"] ;\n6 -> 8 ;\n}" {\n}'